In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup
import re
from random import randint
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pickle
import requests
import math
from calendar import monthrange
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [4]:
def get_table_batch(soup):
    
    party_code = []
    for i, el in enumerate(soup.findAll('div', attrs={'colid':'party_code'})):
        party_code.append(el.text)
    
    party_name = []
    for i, el in enumerate(soup.findAll('div', attrs={'colid':'party_name'})):
        if el.text != 'Name':
            party_name.append(el.text)
    
    cross_party_name = []
    for i, el in enumerate(soup.findAll('div', attrs={'colid':'cross_party_name'})):
        if el.text != 'Cross Party':
            cross_party_name.append(el.text)
    
    rec_date = []
    for i, el in enumerate(soup.findAll('div', attrs={'colid':'rec_date'})):
        if el.text != 'Date':
            rec_date.append(el.text)
    doc_type = []
    for i, el in enumerate(soup.findAll('div', attrs={'colid':'doc_type'})):
        if el.text != 'Type':
            doc_type.append(el.text)
    
    book = []
    for i, el in enumerate(soup.findAll('div', attrs={'colid':'book'})):
        if el.text != 'Book':
            book.append(el.text)
    
    page = []
    for i, el in enumerate(soup.findAll('div', attrs={'colid':'page'})):
        if el.text != 'Page':
            page.append(el.text)
    
    legal_1 = []
    for i, el in enumerate(soup.findAll('div', attrs={'colid':'legal_1'})):
        if el.text != 'Legal':
            legal_1.append(el.text)
    
    file_num = []
    for i, el in enumerate(soup.findAll('div', attrs={'colid':'file_num'})):
        if el.text != 'Instr#':
            file_num.append(el.text)
    
    doc_status = []
    for i, el in enumerate(soup.findAll('div', attrs={'colid':'doc_status'})):
        if el.text != 'Status':
            doc_status.append(el.text)
    
    batch = pd.DataFrame({'Party Code':party_code[1:],
                          'Party Name':party_name,
                          'Cross Party Name':cross_party_name,
                          'Date':rec_date,
                          'Doc Type':doc_type,
                          'Book':book,
                          'Page':page,
                          'Legal':legal_1,
                          'Instr#':file_num,
                          'Doc Status':doc_status})
    return batch

In [5]:
def scroll_and_scrape(returned_rows):
    table = pd.DataFrame([])
    for i in range(0,50):
        try:
            browser.implicitly_wait(1)
            if i == 0:
                browser.implicitly_wait(3)
                html = browser.page_source
                soup = BeautifulSoup(html, 'html.parser')
                table = pd.concat([table, get_table_batch(soup)], axis=0)
                browser.implicitly_wait(1)
                try:
                    row_xpath = '//*[@id="center"]/div/div[4]/div[2]/div/div/div[10]/div[3]'
                    element = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, row_xpath)))
                    element.click()
                except:
                    print('except 10th element')
                    browser.implicitly_wait(1)
                    row = randint(3,10)
                    row_xpath = '//*[@id="center"]/div/div[4]/div[2]/div/div/div[{}]/div[3]'.format(row)
                    element = WebDriverWait(browser, 6).until(EC.element_to_be_clickable((By.XPATH, row_xpath)))
                    element.click()

                actions = ActionChains(browser)
                for _ in range(3):
                    actions.send_keys(Keys.PAGE_DOWN).perform()
                    sleep(0.2)
            else:
                browser.implicitly_wait(0.5)
                try:
                    row_xpath = '//*[@id="center"]/div/div[4]/div[2]/div/div/div[30]/div[3]'
                    element = WebDriverWait(browser, 3).until(EC.element_to_be_clickable((By.XPATH, row_xpath)))
                    element.click()
                except:
                    browser.implicitly_wait(1)
                    row = randint(10,20)
                    row_xpath = '//*[@id="center"]/div/div[4]/div[2]/div/div/div[{}]/div[3]'.format(row)
                    element = WebDriverWait(browser, 5).until(EC.element_to_be_clickable((By.XPATH, row_xpath)))
                    element.click().click()
                actions = ActionChains(browser)
                for _ in range(2):
                    actions.send_keys(Keys.PAGE_DOWN).perform()
                    sleep(0.2)

            if i%2==0:
                browser.implicitly_wait(1)
                sleep(0.1) 
                html = browser.page_source
                soup = BeautifulSoup(html, 'html.parser')
                table = pd.concat([table, get_table_batch(soup)], axis=0)
                dupl = table[table.duplicated(subset=table.columns, keep='first')]

            clean_table = table.drop_duplicates(subset=table.columns, keep='first').copy()
            if clean_table.shape[0] >= 0.99*returned_rows:
                break
        except:
            pass
    return clean_table

## Scraping loop

In [6]:
start_date_path = '/html/body/div[1]/div[3]/div/div[1]/div/div/div/div[1]/form/div[4]/div[2]/p/input'
end_date_path = '/html/body/div[1]/div[3]/div/div[1]/div/div/div/div[1]/form/div[4]/div[3]/p/input'
search_path = '/html/body/div[1]/div[3]/div/div[1]/div/div/div/div[1]/form/div[1]/div[3]/button[2]'
back_search_path = '/html/body/div[1]/div[3]/ul/li[1]/a'

In [7]:
## Selenium
browser = webdriver.Chrome('/Users/macbook/Downloads/chromedriver')

In [ ]:
url = 'http://rodweb.bcgov.net/BrowserViewDMP/'
browser.get(url)

In [ ]:
owner_path = '/html/body/div[1]/div[3]/div/div[1]/div/div/div/div[1]/form/div[1]/div[2]/div/input'
browser.find_element_by_xpath(owner_path).send_keys('MARRIOTT OWNERSHIP RESORTS INC')

In [ ]:
# document type
type_path = '/html/body/div[1]/div[3]/div/div[1]/div/div/div/div[1]/form/div[2]/div[2]/input'
browser.find_element_by_xpath(type_path).send_keys('DEED,RED,REDC,REDH,REDM,TSD,TSDHH,TSDC')

In [1]:
#marriott_table=pd.read_csv('Marriott.csv', index_col=[0])
#marriott_table.tail(3)

In [ ]:
marriott_table.shape

In [ ]:
%%time
for year in range(2009,2021):
    year = str(year)
    for month_num in range(1,13):
        month_table = pd.DataFrame([])
        date_start = '01'
        end_of_month = str(monthrange(int(year), month_num)[1])
        comul_rows = 0
        for i in range(30): 
            #print('\n')
            print('Batch #'+str(i))
           
            if i == 0:
                date_end = str(monthrange(int(year), month_num)[1])
            
            if len(str(month_num)) < 2:
                month = '0' + str(month_num)
            else:
                month = str(month_num)
            returned_rows=1001
            
            # update end and start date       
            browser.find_element_by_xpath(start_date_path).send_keys('{}/{}/{}'.format(month, date_start, year)) 
            browser.find_element_by_xpath(end_date_path).send_keys('{}/{}/{}'.format(month, date_end, year))

            # click search
            sleep(0.3)
            browser.find_element_by_xpath(search_path).click()

            try:
                sleep(0.5)
                ret_rows_path = '//*[@id="results"]/div[6]/div/div[2]'
                element = WebDriverWait(browser, randint(1,4)).until(EC.element_to_be_clickable((By.XPATH, ret_rows_path)))
                returned_rows = element.text
                returned_rows = int(re.findall(r'(\d+(?= total))', returned_rows)[0])
                comul_rows += returned_rows
                #print('Comul Rows:', comul_rows)
                print('Month', month, 'Date Start:', date_start, 'Date End:', date_end, 'Returned Rows:', returned_rows)

                if date_start == '01' and date_end == end_of_month:
                    rows_month_total = returned_rows
                
            except:
                try:
                    sleep(randint(2,3))
                    #browser.implicitly_wait(3)
                    popup = browser.switch_to.alert
                    returned_rows = int(re.findall(r'(\d+(?= results))', popup.text)[0])
                    print('Month', month, 'Date Start:', date_start, 'Date End:', date_end, 'Returned Rows:', returned_rows)
                    if date_start == '01' and date_end == end_of_month: 
                        rows_month_total = returned_rows
                    
                except:
                    not_found = browser.find_element_by_xpath('//*[@id="notFound"]/div/div/p').text

                    if not_found:
                        print('No records')
                        # back to search
                        sleep(0.3)
                        browser.find_element_by_xpath(back_search_path).click()

                        # clear end and start date
                        browser.implicitly_wait(0.3)
                        browser.find_element_by_xpath(end_date_path).clear()
                        browser.find_element_by_xpath(start_date_path).clear()
                        break

            while returned_rows > 1000:
                try:
                    try:
                        browser.implicitly_wait(3)
                        alert = browser.switch_to_alert()
                        alert.accept()
                    except:
                        continue
                    try:
                        # back to search
                        element = WebDriverWait(browser, 3).until(EC.element_to_be_clickable((By.XPATH, back_search_path)))
                        element.click()
                    except:
                        continue

                    # clear end date
                    browser.implicitly_wait(1)
                    browser.find_element_by_xpath(end_date_path).clear()

                    # end date chuncking, update end date
                    sleep(1)
                    if len(date_start)>1 and date_start[0]==0:
                        date_start_num = int(date_start[1])
                    else:
                        date_start_num = int((date_start))

                    if len(date_end)>1 and date_end[0]==0:
                        date_end_num = int(date_end[1])
                    else:
                        date_end_num = int((date_end))

                    date_end = math.trunc(date_start_num + (date_end_num-date_start_num)/(returned_rows/1000))
                    date_end = str(date_end)

                    
                    if len(date_end) < 2:
                        date_end = '0' + date_end

                    browser.find_element_by_xpath(end_date_path).send_keys('{}/{}/{}'.format(month,date_end,year))

                    # click search
                    browser.find_element_by_xpath(search_path).click()
                except:
                    continue

                # check updated results
                try:
                    sleep(randint(2,3))
                    popup = browser.switch_to.alert
                    returned_rows = int(re.findall(r'(\d+(?= results))', popup.text)[0])
                    print('Date Start:', date_start, ' Updated Date End:', date_end, 'Returned rows:', returned_rows)

                except:
                    try:
                        returned_rows = browser.find_element_by_xpath('//*[@id="results"]/div[6]/div/div[2]').text
                        returned_rows = int(re.findall(r'(\d+(?= total))', returned_rows)[0])
                        comul_rows += returned_rows
                        print('Comul Rows:', comul_rows)
                        print('Date Start:', date_start, ' Updated Date End:', date_end, 'Returned rows:', returned_rows)
                        
                    except:
                        returned_rows = 999
                        not_found = browser.find_element_by_xpath('//*[@id="notFound"]/div/div/p').text
                        if not_found:
                            returned_rows = 999
                            pass

            # if <1000 on the first step
            else:
                if returned_rows < 60:
                    sleep(0.5)
                    html = browser.page_source
                    soup = BeautifulSoup(html, 'html.parser')
                    batch = get_table_batch(soup)
                    month_table = pd.concat([month_table, batch], axis=0)

                    # back to search
                    browser.implicitly_wait(1.5)
                    browser.find_element_by_xpath(back_search_path).click()

                    # clear end and start date
                    browser.implicitly_wait(0.3)
                    browser.find_element_by_xpath(end_date_path).clear()
                    browser.find_element_by_xpath(start_date_path).clear()
                    
                    break
                
                else:
        
                    batch = scroll_and_scrape(returned_rows)
                    print('Batch Rows:', batch.shape[0], 'Percent of records: {}%'.format(round(batch.shape[0]/returned_rows*100, 1)))
                    sleep(0.5) 
                    month_table = pd.concat([month_table, batch], axis=0)
                    month_table['Start Date'] = date_start
                    month_table['End Date'] = date_end
                    month_table['Percentage of data'] = round(batch.shape[0]/returned_rows*100, 1)

                    date_start = str(int(date_end)+1)

                    #EDIT
                    if date_end==end_of_month: 
                    #print('Final Step', 'Rows Month Total:', rows_month_total)
                        print('Month table rows:', month_table.shape[0], 'Percent of total: {}%'.format(round(month_table.shape[0]/rows_month_total*100, 1))) 
                        # back to search
                        sleep(0.3)
                        browser.implicitly_wait(1)
                        browser.find_element_by_xpath(back_search_path).click()

                        # clear end and start date
                        browser.implicitly_wait(0.3)
                        browser.find_element_by_xpath(end_date_path).clear()
                        browser.find_element_by_xpath(start_date_path).clear()

                        break

                    date_end = math.trunc(int(date_start) + (int(end_of_month)-int(date_start))/((rows_month_total-comul_rows*0.99)/1000))
                    if date_end > int(end_of_month):
                        date_end = end_of_month
                    date_end = str(date_end)
                    if len(date_end) < 2:
                            date_end = '0' + date_end

                    if len(date_start) < 2:
                        date_start = '0' + date_start


                    if len(date_start) < 2:
                        date_start = '0' + date_start

                    # back to search
                    sleep(0.3)
                    browser.implicitly_wait(1)
                    browser.find_element_by_xpath(back_search_path).click()

                    # clear end and start date
                    browser.implicitly_wait(0.3)
                    browser.find_element_by_xpath(end_date_path).clear()
                    browser.find_element_by_xpath(start_date_path).clear()

        
        #marriott_table = pd.concat([marriott_table, month_table], axis=0)
        #marriott_table.to_csv('----.csv')
        print('Marriott Table Rows:', marriott_table.shape[0])

In [ ]:
#year_table_final.to_csv('timeshares_{}'.format('2012'))

In [ ]:
year_table_final.shape

In [ ]:
year_table.drop_duplicates(subset=year_table.columns[:8], keep='first').shape